In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from multicall import Call

from mainnet_launch.abis import CURVE_STABLESWAP_NG_ABI
from mainnet_launch.constants import AUTO_ETH, ETH_CHAIN
from mainnet_launch.data_fetching.alchemy.get_events import fetch_events, get_each_event_in_contract
from plotly.subplots import make_subplots
from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    safe_normalize_with_bool_success,
    identity_with_bool_success,
    safe_normalize_6_with_bool_success,
    build_blocks_to_use
)

import plotly.graph_objects as go
pio.templates.default = "plotly_white"

POOL_ADDRESS = "0x5B03CcCAb7BA3010fA5CAd23746cbf0794938e96"  # Curve USDT-USDE pool
CSV_PATH = "./curve_usdt_usde_destination_states.csv"

df = pd.read_csv(CSV_PATH)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.set_index("datetime")

df = df[df.index > "2025-10-01"].copy()

vp_call = Call(
    target=POOL_ADDRESS,
    function=["get_virtual_price()(uint256)"],
    returns=[("virtual_price", safe_normalize_with_bool_success)],
)

usdt_call = Call(
    target=POOL_ADDRESS,
    function=["balances(uint256)(uint256)", 0],
    returns=[("usdt", safe_normalize_6_with_bool_success)],
)

usde_call =Call(
    target=POOL_ADDRESS,
    function=["balances(uint256)(uint256)", 1],
    returns=[("usde", safe_normalize_with_bool_success)],
)

stored_rates_call = Call(
    target=POOL_ADDRESS,
    function=["stored_rates()(uint256[])"],
    returns=[("stored_rates", identity_with_bool_success)],
)

total_supply_call = Call(
    target=POOL_ADDRESS,
    function=["totalSupply()(uint256)"],
    returns=[("total_supply", safe_normalize_with_bool_success)],
)

# Remove the individual usdt_rate and usde_rate calls
# We'll extract them from the stored_rates result after fetching
blocks = build_blocks_to_use(ETH_CHAIN)

onchain_df = get_raw_state_by_blocks(
    calls=[vp_call, usdt_call, usde_call, stored_rates_call, total_supply_call],
    blocks= blocks,
    chain=AUTO_ETH.chain,
    include_block_number=True,
)


onchain_df['percent_usdt'] = 100 * onchain_df['usdt'] / (onchain_df['usdt'] + onchain_df['usde'])
# guard against division-by-zero / NaN
onchain_df['percent_usdt'] = onchain_df['percent_usdt'].fillna(0)
onchain_df['naive_size'] = onchain_df['usdt'] + onchain_df['usde']
contract = ETH_CHAIN.client.eth.contract(POOL_ADDRESS, abi=CURVE_STABLESWAP_NG_ABI)
onchain_df['naive_vp'] =   onchain_df['naive_size'] / onchain_df['total_supply']


events = get_each_event_in_contract(contract, start_block=df["block"].min(), end_block=df["block"].max(), chain=ETH_CHAIN)
onchain_df.head()


/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/eth_abi/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


,virtual_price,usdt,usde,stored_rates,total_supply,block,percent_usdt,naive_size,naive_vp
timestamp,,,,,,,,,
2024-09-10 23:59:11+00:00,NaN,NaN,NaN,None,NaN,20723645,0.0,NaN,NaN
2024-09-11 23:57:11+00:00,NaN,NaN,NaN,None,NaN,20730797,0.0,NaN,NaN
2024-09-12 22:57:35+00:00,NaN,NaN,NaN,None,NaN,20737657,0.0,NaN,NaN
2024-09-13 23:58:11+00:00,NaN,NaN,NaN,None,NaN,20745124,0.0,NaN,NaN
2024-09-14 23:58:35+00:00,NaN,NaN,NaN,None,NaN,20752280,0.0,NaN,NaN


# our fee + base apr for curve is 


look at the virtual price 5 days ago, 

compute the growth rate between vp[t-5], vp[t],

project that into the future. 

those come with expectations about the apr

In [ ]:
import numpy as np
import pandas as pd

def trimmed_mean_of_step_changes(window: np.ndarray, trim: int = 3) -> float:
    """
    window: length 16
    compute 15 step changes: w[1]-w[0], ..., w[15]-w[14]
    drop trim smallest and trim largest changes
    return mean of remaining
    """
    w = np.asarray(window, dtype=float)

    diffs = np.diff(w)  # length 15
    diffs.sort()

    return diffs[trim:-trim].mean()



def run_our_virtual_price_forcast(onchain_df, lookback_days, lookforward_days):
    vp_df = onchain_df[['virtual_price']].dropna().rename(columns={'virtual_price': 'current_vp'}).sort_index()
    vp_df['prior_vp']  = vp_df['current_vp'].shift(lookback_days)
    vp_df['future_vp'] = vp_df['current_vp'].shift(-lookforward_days)

    portion_change = 

    vp_df['actual_one_day_look_forward'] = ((vp_df['future_vp'] -  vp_df['current_vp']  ) / vp_df['current_vp'] ) / 365
    vp_df['actual_ann_return'] = 100 * ((1 + vp_df['actual_lf_return']) ** (365 / lookforward_days) - 1)

    # # Trailing lookback growth factor
    # vp_df['g_lb'] = vp_df['current_vp'] / vp_df['prior_vp']

    # # Expected future VP using implied per-day compounding
    # vp_df['expected_vp'] = vp_df['current_vp'] * (vp_df['g_lb'] ** (lookforward_days / lookback_days))


    predicted_mean_change = vp_df['current_vp'].rolling(window=lookback_days, min_periods=lookback_days).apply(
        lambda w: trimmed_mean_of_step_changes(w, trim=3),
        raw=True,  # passes a numpy array to the function (faster)
    )

    vp_df['expected_vp'] = vp_df['current_vp'] + ((1 + predicted_mean_change) ** (lookforward_days / 365))

    # Expected forward lf-day return / annualized
    vp_df['expected_lf_return'] = vp_df['expected_vp'] / vp_df['current_vp'] - 1
    vp_df['expected_ann_return'] = 100 * ((1 + vp_df['expected_lf_return']) ** (365 / lookforward_days) - 1)

    vp_df['expected_minus_actual'] = vp_df['expected_ann_return'] - vp_df['actual_ann_return']

    # print(f'using lookback_days={lookback_days}, lookforward_days={lookforward_days}')
    # px.scatter(vp_df, y=['future_vp', 'expected_vp', 'current_vp'], title='Actual future vs expected future').show()
    return vp_df

lookback_days = 15
lookforward_days = 30

vp_df = run_our_virtual_price_forcast(onchain_df, lookback_days, lookforward_days)
vp_df

,current_vp,prior_vp,future_vp,actual_lf_return,actual_ann_return,expected_vp,expected_lf_return,expected_ann_return,expected_minus_actual
timestamp,,,,,,,,,
2025-02-13 23:59:59+00:00,1.000019,NaN,1.003190,0.003171,3.927476,NaN,NaN,NaN,NaN
2025-02-14 23:59:59+00:00,1.000163,NaN,1.003285,0.003122,3.864872,NaN,NaN,NaN,NaN
2025-02-15 23:59:59+00:00,1.000271,NaN,1.003456,0.003184,3.943428,NaN,NaN,NaN,NaN
2025-02-16 23:59:59+00:00,1.000293,NaN,1.003634,0.003340,4.140446,NaN,NaN,NaN,NaN
2025-02-17 23:59:59+00:00,1.000323,NaN,1.003811,0.003487,4.326303,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2026-01-10 23:59:59+00:00,1.020751,1.019222,NaN,NaN,NaN,2.020757,0.979677,405936.435794,NaN
2026-01-11 23:59:59+00:00,1.020770,1.019222,NaN,NaN,NaN,2.020777,0.979659,405890.048685,NaN
2026-01-12 23:59:59+00:00,1.020797,1.019223,NaN,NaN,NaN,2.020803,0.979633,405826.519994,NaN


In [172]:
def analyze_prediction_error(vp_df, lookback_days, lookforward_days):
    """
    Analyze prediction error for virtual price forecasting.
    
    Parameters:
    -----------
    vp_df : pd.DataFrame
        DataFrame with 'expected_ann_return', 'actual_ann_return', and 'expected_minus_actual' columns
    lookback_days : int
        Number of days used for lookback window
    lookforward_days : int
        Number of days used for lookforward window
    
    Returns:
    --------
    dict : Dictionary containing error statistics
    """
    print(f'using lookback_days={lookback_days}, lookforward_days={lookforward_days}')
    print('error is expected minus actual')
    # Plot the returns
    px.line(vp_df[['actual_ann_return', 'expected_ann_return']], title='Expected vs Actual Annualized Return').show()
    
    # Calculate absolute mean error
    absolute_mean_error = vp_df['expected_minus_actual'].abs().mean().round(2)
    print(f'absolute mean error: {absolute_mean_error}%')
    
    # Display absolute quantiles
    quantiles = vp_df['expected_minus_actual'].abs().quantile([0.80, 0.90, 0.95])
    print(f"\n80th percentile: {quantiles[0.80]:.2f}%")
    print(f"90th percentile: {quantiles[0.90]:.2f}%")
    print(f"95th percentile: {quantiles[0.95]:.2f}%")
    
    # Calculate directional mean error
    mean_error = vp_df['expected_minus_actual'].mean().round(2)
    print(f'\ndirectioned mean error: {mean_error}%')
    
    # Display directional quantiles
    directioned_quantiles = vp_df['expected_minus_actual'].quantile([0.05, 0.10, 0.20, 0.80, 0.90, 0.95])
    print(f"\n5th percentile: {directioned_quantiles[0.05]:.2f}%")
    print(f"10th percentile: {directioned_quantiles[0.10]:.2f}%")
    print(f"20th percentile: {directioned_quantiles[0.20]:.2f}%")
    print(f"80th percentile: {directioned_quantiles[0.80]:.2f}%")
    print(f"90th percentile: {directioned_quantiles[0.90]:.2f}%")
    print(f"95th percentile: {directioned_quantiles[0.95]:.2f}%")

    px.scatter(vp_df, x='expected_ann_return', y='actual_ann_return', title='Expected vs Actual Annualized Return Scatter Plot').show()
    
    return {
        'absolute_mean_error': absolute_mean_error,
        'mean_error': mean_error,
        'quantiles_absolute': quantiles,
        'quantiles_directional': directioned_quantiles,
        'lookback_days': lookback_days,
        'lookforward_days': lookforward_days
    }

results = analyze_prediction_error(vp_df, lookback_days, lookforward_days)

using lookback_days=15, lookforward_days=30
error is expected minus actual


AttributeError: 'float' object has no attribute 'round'

In [170]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

# strong prior
- Fee and base APR does not stay high for long
- Fees are genarally near 0 with some unpredictable massive spikes
- Outliers are the most important because we are sorting on apr
- A mean error of 1 with a std of 1 is much better than an mean error of 1 and a std of 2


# one attack vector is I just donate to the pool,

do a bunch of wash trading, as fees to get the virtual price up. that make ssense because I can wash trade away 500 in value to get 500k in value allocated here.




In [ ]:
break

In [38]:
vp_df[['actual_annualized_return', 'expected_annualized_return', 'expected_minus_actual']].describe()

KeyError: "['actual_annualized_return', 'expected_minus_actual'] not in index"

In [ ]:
brealk

In [ ]:
i = 5
vp_df = onchain_df[['virtual_price']].dropna()
vp_df['current_vp'] = vp_df['virtual_price']
vp_df['prior_vp'] = vp_df['virtual_price'].shift(i)
vp_df['future_vp'] = vp_df['virtual_price'].shift(-i)
# our current trend says, past growth rate will continue into the future

vp_df['actual_n_day_return'] = ((vp_df['future_vp'] - vp_df['current_vp']) / vp_df['current_vp'])
vp_df['actual_annualized_return'] = 100 * ((1 + vp_df['actual_n_day_return']) ** (365 / i) - 1)

vp_df['expected_n_day_return'] = (vp_df['current_vp'] - vp_df['prior_vp']) / vp_df['prior_vp']
vp_df['expected_annualized_return'] = 100 * ((1 + vp_df['expected_n_day_return']) ** (365 / i) - 1)
vp_df['actual_minus_expected'] = vp_df['actual_annualized_return'] - vp_df['expected_n_day_return']
vp_df['abs_errror'] = vp_df['actual_minus_expected'].abs()
# negative means under estiamte, positive means over estimate

vp_df = vp_df.dropna()
print(vp_df[['actual_annualized_return', 'expected_annualized_return']].describe().round(2))
px.line(vp_df[['actual_annualized_return', 'expected_annualized_return']], title='Actual vs Expected Annualized Return based on Virtual Price').show()

       actual_annualized_return  expected_annualized_return
count                    326.00                      326.00
mean                       2.32                        2.30
std                        2.53                        2.52
min                        0.01                        0.01
25%                        0.34                        0.34
50%                        1.15                        1.21
75%                        3.73                        3.71
max                       10.61                       10.61


In [ ]:
px.scatter(vp_df, x='expected_annualized_return', y='abs_errror', title='Expected vs Actual Annualized Return based on Virtual Price').show()

In [ ]:
i = 5
vp_df = onchain_df[['virtual_price']].dropna()
prior_vp = vp_df['virtual_price'].shift(i)
vp_df[f'absolute_change_{i}d'] = (vp_df['virtual_price'] - prior_vp) / prior_vp 
vp_df['average_one_day_change'] = vp_df[f'absolute_change_{i}d'] / i
vp_df['expected_vp_5d'] = prior_vp * ((1 + vp_df['average_one_day_change']) ** 5)
vp_df


,virtual_price,absolute_change_5d,average_one_day_change,expected_vp_5d
timestamp,,,,
2025-02-13 23:59:59+00:00,1.000019,NaN,NaN,NaN
2025-02-14 23:59:59+00:00,1.000163,NaN,NaN,NaN
2025-02-15 23:59:59+00:00,1.000271,NaN,NaN,NaN
2025-02-16 23:59:59+00:00,1.000293,NaN,NaN,NaN
2025-02-17 23:59:59+00:00,1.000323,NaN,NaN,NaN
...,...,...,...,...
2026-01-10 23:59:59+00:00,1.020751,0.000715,0.000143,1.020751
2026-01-11 23:59:59+00:00,1.020770,0.000354,0.000071,1.020770
2026-01-12 23:59:59+00:00,1.020797,0.000119,0.000024,1.020797


In [ ]:
px.line(vp_df, y=['virtual_price', 'expected_vp_5d'], title='Virtual Price and our expected VP (assume growth in last 5 days == growth in next 5 days)').show()

In [ ]:
# annualized_return = (end_value / start_value) ** (1 / num_years) - 1

In [ ]:
1.0003250522953844 - 1.0000188969188437, 0.00030614959125668215

(0.00030615537654066216, 0.00030614959125668215)

In [ ]:

    # look back 5 
    


In [ ]:
# Get the starting and ending values of the virtual price

days_before_end = 150
start_value = vp_df['virtual_price'].iloc[0]
end_value = vp_df['virtual_price'].iloc[-days_before_end]

# Calculate the number of years between the start and end timestamps
num_years = (vp_df.index[-days_before_end] - vp_df.index[0]).days / 365.25

# Compute the annualized return
annualized_return = (end_value / start_value) ** (1 / num_years) - 1

print(f"Annualized Performance: {annualized_return:.6%}")
num_years

Annualized Performance: 3.130502%


0.5092402464065708

In [ ]:
px.line(onchain_df, y='virtual_price', title='Virtual Price Over Time')

In [ ]:
AddLiquidity = events['AddLiquidity'].copy()

AddLiquidity['naive_vp'] = AddLiquidity.apply(lambda row: row['invariant'] / row['token_supply'], axis=1)
# ensure rows are ordered by block (earliest/smallest first)
AddLiquidity = AddLiquidity.sort_values("block", ascending=True).reset_index(drop=True)
AddLiquidity['naive_vp_increase'] = AddLiquidity['naive_vp'].diff()
AddLiquidity['usdt_fees'] = AddLiquidity['fees_0'].apply(lambda x: x / 1e6)
AddLiquidity['usde_fees'] = AddLiquidity['fees_1'].apply(lambda x: x / 1e18)
AddLiquidity

,event,block,transaction_index,log_index,hash,provider,token_amounts_0,token_amounts_1,fees_0,fees_1,invariant,token_supply,naive_vp,naive_vp_increase,usdt_fees,usde_fees
0,AddLiquidity,23510309,256,646,0xcd17a9e9522cf0504d2a58ca002cdda71ce7558ad3c8...,0x13B90df23158808185B247aB61EDe61b75d49E23,290226585,0,3381,3226863946983196,684946058565969797567551,672958106681316564153146,1.017814,NaN,0.003381,0.003227
1,AddLiquidity,23516107,85,195,0xda7c954659b33af2ded557126f6b72b0371c714deb1f...,0x7FfB2205119c618661a54Cb10Fa48f8fa61B4bE0,60711218300,0,879738,782704761652758295,745692441351794501615041,732613351699008003609977,1.017853,3.883387e-05,0.879738,0.782705
2,AddLiquidity,23523103,46,150,0x595ffa4efde071c40acda5dbeb647b0e27f82c78dbfc...,0x7a48010f11986be22521548b5e8e127ffe63911C,0,1749217038304634178727,7581,7103807538467950,241690963066401979280471,237405729776159885797934,1.018050,1.976023e-04,0.007581,0.007104
3,AddLiquidity,23527501,200,508,0x66952dec9a1486bb4643e978a9b1fd9383f4dc778b1b...,0x01e0B21888D74901670ACa0c8F3321bFAd14e2F2,3000000000,0,48976,39922069101797232,242946246344107613158133,238635202507544491466501,1.018065,1.516228e-05,0.048976,0.039922
4,AddLiquidity,23527820,127,227,0x7e9b6b29b56fe61e8025e72985ac8092b606407431c0...,0x01e0B21888D74901670ACa0c8F3321bFAd14e2F2,13984817691,0,210706,182312395580522394,256933676387225633749987,252374381573386593294549,1.018066,1.862836e-07,0.210706,0.182312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,AddLiquidity,24190390,93,316,0xcaa98bf8e06e2f3aa108d16c428e140fd7f123952dfd...,0xcc97cc6B9DeFB70838151711c47f3834b8c97919,37356386431,7319653222969362628491,24,30797555639548,663288212342899149071903,649831049195195947615230,1.020709,2.082489e-06,0.000024,0.000031
58,AddLiquidity,24192182,7,152,0x0ebbb9319a0a600f32745b7d47f00ab234e1cb5942ed...,0xcc97cc6B9DeFB70838151711c47f3834b8c97919,27954942629,5549737241233547843598,18,23272190245699,663365482055566252607698,649906532876023570876906,1.020709,3.429141e-07,0.000018,0.000023
59,AddLiquidity,24204108,316,625,0xd018cfdc189c710fcc63a17deb6a92968e4adb551ce3...,0x4Fe93ebC4Ce6Ae4f81601cC7Ce7139023919E003,822894620,0,2297,2792855244037337,630932593531059397467573,618108506959713646147933,1.020747,3.825285e-05,0.002297,0.002793
60,AddLiquidity,24205702,123,225,0x4b20ab71ffb5880e9890287626172ae06fa5e5fce97c...,0x56C526b0159a258887e0d79ec3a80dfb940d0cD7,386102702,864791461071045716107,10333,12599370720082281,632185465409920195427705,619334155408853344946737,1.020750,2.897508e-06,0.010333,0.012599


In [ ]:
events['RemoveLiquidityOne']

,event,block,transaction_index,log_index,hash,provider,token_id,token_amount,coin_amount,token_supply
0,RemoveLiquidityOne,23500925,67,483,0x28fc75c5a25e44157b530a1f345a4fec935e02682f77...,0xCbE33E103a9BFE193609e8435970C138b4b10A2A,0,98352888467144058079,100059272,672672942307688235689845
1,RemoveLiquidityOne,23517943,71,176,0xf6741d6e9fc6af554c4e60ad9a7ab34b6276cbddd5bb...,0x286efb152c50496d2cED0a77Cd76dD55af6025a9,0,1000000000000000000000,1017807473,731613351699008003609977
2,RemoveLiquidityOne,23517958,107,126,0xbc531c471704dd15bd688343ac454ae7bfa7e5affb13...,0x286efb152c50496d2cED0a77Cd76dD55af6025a9,0,100000000000000000000000,101770694992,631613351699008003609977
3,RemoveLiquidityOne,23517964,43,219,0x05d0969a4da47cabeae2737fc62940f1a474dd3f62e7...,0x286efb152c50496d2cED0a77Cd76dD55af6025a9,0,100000000000000000000000,101768248433,531613351699008003609977
4,RemoveLiquidityOne,23517971,21,169,0xc94498cef1876ce14a9b0696ccdb4fb45234008d211b...,0x286efb152c50496d2cED0a77Cd76dD55af6025a9,0,100000000000000000000000,101684123181,431613351699008003609977
5,RemoveLiquidityOne,23518044,48,75,0x505767582495ce229cee54148a93dc439c3d1fa410a6...,0x286efb152c50496d2cED0a77Cd76dD55af6025a9,0,100000000000000000000000,101713737740,331613351699008003609977
6,RemoveLiquidityOne,23519698,155,429,0xbd39d82afc043c2a20c92d7dd76837c13937da55ce29...,0x286efb152c50496d2cED0a77Cd76dD55af6025a9,0,95925712338774770294768,97643284976,235687639360233233315209
7,RemoveLiquidityOne,23523120,92,148,0x7519bdd75feca38641d30df67a0f0943b8c43ee1300a...,0x7a48010f11986be22521548b5e8e127ffe63911C,0,1718090415926652482725,1748869615,235687639360233233315209
8,RemoveLiquidityOne,23531562,75,328,0x0600415260daea025a15eb7ef974f01688bb6622e120...,0x01e0B21888D74901670ACa0c8F3321bFAd14e2F2,0,16686742213153359979340,16985229754,235687639360233233315209
9,RemoveLiquidityOne,23539707,168,520,0x5f8cd1562dba81d29169c30b38592e5eec63029800ff...,0x7FfB2205119c618661a54Cb10Fa48f8fa61B4bE0,0,48500000000000000000000,49386174994,227241889048131098502549
